In [6]:
import numpy as np
from PIL import Image
import torch
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity as lpips
from torchvision import transforms
from SIT import *


In [7]:
epsilon = 16/255
alpha = 1.6/255
epochs = 10
decay = 1

lpips_loss = lpips(net_type='squeeze', normalize=True)

In [8]:
def add_noise(x, noise=None):
        if noise != None:
               return torch.clip(x + noise, 0, 1)
        return torch.clip(x + torch.zeros_like(x).uniform_(-16/255,16/255), 0, 1)

def tensor_to_image(image):
      img = image.permute(1, 2, 0).cpu().detach().numpy() * 255
      img = img.astype(np.uint8)
      img = Image.fromarray(img)
      return img

def init_delta(data, method=None):
      if method == None:
            delta = torch.zeros_like(data)
            delta.uniform_(-epsilon, epsilon)
            delta.requires_grad = True
            return delta
      elif method == "SIT":
        data_copy = blocktransform(data)
        copy_img = tensor_to_image(data_copy[0])
        copy_img.show()
        delta = data_copy - data
      
      return delta

def get_loss(image1, image2):
       res = lpips_loss(image1, image2)
       return res

def update_delta(delta, grad):
      delta = torch.clamp(delta + alpha * grad.sign(), -epsilon, epsilon)
      return delta
       

image_path = "dog.jpg"

transform = transforms.Compose([
  transforms.ToTensor()  # Convert to tensor with pixel values between 0 and 1
])

In [9]:
image = Image.open(image_path).convert('RGB')

transformed_image = transform(image)
transformed_image.requires_grad = True

transformed_image = transformed_image.view(1,3,1000,1500)

In [10]:
delta = init_delta(transformed_image, method="None")
# delta = init_delta(transformed_image)
noised_image = add_noise(transformed_image, delta)
noised_image = noised_image.view(1,3,1000,1500)

momentum = 0.
for _ in range(10):
        loss = get_loss(transformed_image, noised_image)
        grad = torch.autograd.grad(loss, delta)[0]
        momentum = momentum * decay + grad / (grad.abs().mean(dim=(1,2,3), keepdim=True))
        delta = update_delta(delta, momentum)
        noised_image = add_noise(noised_image, delta)
        new_img = tensor_to_image(noised_image[0])
        new_img.save(f'LPIPS_Images/Lpips_image_{_}.jpg')
        print(loss)

UnboundLocalError: cannot access local variable 'delta' where it is not associated with a value